# Windshield

#### Steps
1. Load packages
2. Load images **
3. Set image to gray scale
4. Threshold image
5. Gaussian Blur
6. Canary Edge detection
7. Find contours
8. Find contour density
9. Hough Line Transform  
10. Thresholds for shattered glass detection
11. Save image

In [ ]:
import cv2
import numpy as np
import os

def process_and_analyze_image(file_path, min_contour_density=0.0001, min_line_count=90):
    """
    Processes an image to analyze contours, detect lines, and identify potential damage based on the file path.

    Parameters:
        file_path (str): Path to the image file.
        min_contour_density (float): Threshold for contour density to classify as damaged.
        min_line_count (int): Threshold for line count to classify as damaged.

    Returns:
        contour_image (np.array): Image with contours and lines drawn.
        result_text (str): Result message indicating damage status.
    """

    # Read the original image in grayscale
    original_image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    if original_image is None:
        print("Error: Image not found or invalid file path.")
        return None, None

    # Apply binary thresholding
    _, binary_image = cv2.threshold(original_image, 130, 255, cv2.THRESH_BINARY)

    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(binary_image, (5, 5), 0)

    # Detect edges using Canny edge detection
    edges = cv2.Canny(blurred, 50, 110)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_count = len(contours)
    image_area = original_image.shape[0] * original_image.shape[1]
    contour_density = contour_count / image_area
    print(f"Contour Count: {contour_count}")
    print(f"Contour Density: {contour_density:.6f}")

    # Sort contours by area in descending order
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Convert edge image to BGR for visualization
    contour_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Draw contours on the image
    cv2.drawContours(contour_image, sorted_contours, -1, (0, 255, 0), 2)  # Green contours

    # Detect lines using Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)
    line_count = len(lines) if lines is not None else 0

    # Draw Hough lines on the image
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(contour_image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue lines

    # Shattered glass detection logic
    if contour_density > min_contour_density and line_count > min_line_count:
        result_text = "Shattered Glass Detected"
        print("Shattered glass detected! This will take 1 hour to fix.")
    else:
        result_text = "No Damage Detected"
        print("No shattered glass detected.")

    # Add result text to the image
    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(contour_image, result_text, (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Generate output filename based on the input filename
    base_name = os.path.splitext(os.path.basename(file_path))[0]  # Get the base filename without extension
    output_filename = f"{base_name}_processed.jpg"  # Add '_processed' to the base name

    # Save the processed image
    cv2.imwrite(output_filename, contour_image)
    print(f"Processed image saved as {output_filename}")

    return contour_image, result_text


#Usage
#file_path = '/Users/ryanalbelda/Documents/PyCharm and Python Home Folder/Computer_Vision/Project/windsheild3.jpg'
#file_path = '/Users/ryanalbelda/Documents/PyCharm and Python Home Folder/Computer_Vision/Project/windsheild4.jpg'
#file_path = '/Users/ryanalbelda/Documents/PyCharm and Python Home Folder/Computer_Vision/Project/windsheild6.png'
#file_path = '/Users/ryanalbelda/Documents/PyCharm and Python Home Folder/Computer_Vision/Project/windsheild7.png'
#file_path = '/Users/ryanalbelda/Documents/PyCharm and Python Home Folder/Computer_Vision/Project/windsheild8.jpg'
#file_path = '/Users/ryanalbelda/Documents/PyCharm and Python Home Folder/Computer_Vision/Project/challange_image.jpg'

processed_image, message = process_and_analyze_image(file_path)

# Display the result
if processed_image is not None:
    cv2.imshow('Processed Image', processed_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Contour Count: 767
Contour Density: 0.004607
Shattered glass detected! This will take 1 hour to fix.
Processed image saved as windsheild8_processed.jpg
